## A Measure of the Complexity of Neural Representations based on Partial Information Decomposition
David A. Ehrlich, Andreas C. Schneider, Viola Priesemann, Michael Wibral, Abdullah Makkeh. TMLR 2023.\
Supplementary Code -  Script 4/5
### Training and evaluating the convolutional partly quantized, one-hot-output CIFAR network

In [ ]:
%load_ext autoreload
%autoreload 2

### Train network

In [ ]:
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
cm = 1/2.54  # centimeters to inches

import nninfo

In [ ]:
# Set experiment id
experiment_id = "cifar10"

#### Set up parameters and train first network

In [ ]:
# Note that we do not set initial seeds manually here, but save all seeds to the
# checkpoints files during training for later reproducibility. Rerunning this script
# will produce slightly different figures due to the randomness of network
# initialization etc.

# Set network architecture
# Input size is 3 * 32 * 32
# Then three convolutional layers with 32, 64 and 128 filters
# Then a fully connected network part with 2048, 128, 32, 5, 5 and finally 10 neurons
layer_infos = [
    nninfo.LayerInfo(connection_layer='input', activation_function='input'),

    nninfo.LayerInfo(connection_layer='conv2d', connection_layer_kwargs={'in_channels': 3, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': (1, 1)}, activation_function='relu'),
    nninfo.LayerInfo(connection_layer='maxpool2d', connection_layer_kwargs={'kernel_size': 2}, activation_function=None),
    
    nninfo.LayerInfo(connection_layer='conv2d', connection_layer_kwargs={'in_channels': 32, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': (1, 1)}, activation_function='relu'),
    nninfo.LayerInfo(connection_layer='maxpool2d', connection_layer_kwargs={'kernel_size': 2}, activation_function=None),

    nninfo.LayerInfo(connection_layer='conv2d', connection_layer_kwargs={'in_channels': 32, 'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': (1, 1)}, activation_function='relu'),
    nninfo.LayerInfo(connection_layer='maxpool2d', connection_layer_kwargs={'kernel_size': 2}, activation_function=None),

    nninfo.LayerInfo(connection_layer='flatten', activation_function=None),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 1024, 'out_features': 2048}, activation_function='tanh'),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 2048, 'out_features': 128}, activation_function='tanh'),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 128, 'out_features': 32}, activation_function='tanh'),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 32, 'out_features': 5}, activation_function='tanh'),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 5, 'out_features': 5}, activation_function='tanh'),
    nninfo.LayerInfo(connection_layer='linear', connection_layer_kwargs={'in_features': 5, 'out_features': 10}, activation_function='softmax_output'),
]

# Set weight initialization
initializer_name = 'xavier'


# Create network instance
network = nninfo.NeuralNetwork(layer_infos=layer_infos,
                               init_str=initializer_name)

# Set task instance
task = nninfo.TaskManager('cifar10_1d_dat')
# Split into train and test set
task['full_set'].train_test_val_sequential_split(50000, 10000, 0)

# Create quantizer list with stochastic quantization. The input and convolutional layers are not quantized.
quantizer = 8 * [None] + 5 * [{'levels': 8, 'rounding_point': 'stochastic'}] + [None]

# Initialize training components
trainer = nninfo.Trainer(dataset_name='full_set/train',
                         optim_str='SGD',
                         loss_str='CELoss',
                         lr=0.005,
                         shuffle=True,
                         batch_size=64,
                         quantizer=quantizer)

tester = nninfo.Tester(dataset_name='full_set/test')

schedule = nninfo.Schedule()

# Save training state for 30 logarithmically spaced checkpoints
schedule.create_log_spaced_chapters(1000, 30)

# Combine components into experiment
experiment = nninfo.Experiment(experiment_id=experiment_id,
                        network=network,
                        task=task,
                        trainer=trainer,
                        tester=tester,
                        schedule=schedule)

# Run training for 10^5 epochs
experiment.run_following_schedule()

In [ ]:
# Set up experiment
exp = nninfo.exp.Experiment(experiment_id, load=True)

# Compute 9 more training runs with different random weight initializations.
exp.rerun(9)

### Evaluate network performance
#### Compute loss and accuracy

In [ ]:
quantizer_params = 8 * [None] + 5 * [{'levels': 8, 'rounding_point': 'center_saturating'}] + [None]

experiment = nninfo.Experiment.load(experiment_id)
performance_measurement = nninfo.analysis.PerformanceMeasurement(experiment, ['full_set/train', 'full_set/test'], quantizer_params=quantizer_params)

performance_measurement.perform_measurements(run_ids='all', chapter_ids='all', exists_ok=True)

In [ ]:
# Load performance file
experiment = nninfo.Experiment.load(experiment_id)
performance_measurement = nninfo.analysis.PerformanceMeasurement.load(experiment, "performance")

fig, ax = plt.subplots(figsize=(4*cm, 4*cm), dpi=150)
ax.set_ylim(0, 1)
twinax = ax.twinx()

# Plot accuracy
nninfo.plot.plot_loss_accuracy(performance_measurement.results, ax, twinax)

ax.legend(ncol=1, bbox_to_anchor=(1.5, 0.5), loc='center left');

In [ ]:
# Save result
plt.savefig(f"experiments/exp_{experiment_id}/plots/performance.pdf", bbox_inches='tight');

### Perform quintivariate PID on hidden layers $L_3$, $L_4$ and $L_5$

In [ ]:
# Load experiment
exp = nninfo.Experiment.load(experiment_id)

for layer in [10, 11]:

    target = [nninfo.NeuronID('Y', (1,))]
    source1 = [nninfo.NeuronID(f'L{layer}', (1,))]
    source2 = [nninfo.NeuronID(f'L{layer}', (2,))]
    source3 = [nninfo.NeuronID(f'L{layer}', (3,))]
    source4 = [nninfo.NeuronID(f'L{layer}', (4,))]
    source5 = [nninfo.NeuronID(f'L{layer}', (5,))]

    # Create quantizer list for deterministic rounding.
    quantizer = 8 * [None] + 5 * [{'levels': 8, 'dequant_point': 'center_saturating'}]

    # Compute PID for all chapters of all random network initializations
    pid_measurement = nninfo.analysis.PIDMeasurement(experiment,
                                                     measurement_id=f'pid_L{layer}',
                                                     dataset_name='full_set/train',
                                                     pid_definition='sxpid',
                                                     target_id_list=target,
                                                     source_id_lists=[source1, source2, source3, source4, source5],
                                                     binning_kwargs={'binning_method':'none'},
                                                     quantizer_params=quantizer_params)

    itic = time.time_ns()
    pid_measurement.perform_measurements(run_ids='all', chapter_ids='all')
    itoc = time.time_ns()
    print(f"Computing PID for L{layer} took: ", (itoc-itic)/10**9, "s")

### Compute representational complexity and plot results

In [ ]:
from nninfo.postprocessing.pid_postprocessing import get_pid_summary_quantities

cm = 1/2.54
fig, ax= plt.subplots(figsize=(5*cm, 4*cm), dpi=150)
inset_axis = inset_axes(ax, width=0.7, height=0.4)

# Load experiment
experiment = nninfo.experiment.Experiment.load(experiment_id)

# Plot accuracy on inset axis
performance_measurement = nninfo.analysis.PerformanceMeasurement.load(experiment, 'performance')
nninfo.plot.plot_accuracy(performance_measurement.results, 'full_set/train', inset_axis, c='k', label='Train')
nninfo.plot.plot_accuracy(performance_measurement.results, 'full_set/test', inset_axis, c='k', ls='--', label='Test')
inset_axis.set_ylim(0, 1)
inset_axis.set_ylabel('Acc.')
inset_axis.legend(bbox_to_anchor=(1.1, 1.3), loc='upper left')
    
for layer in [10, 11]:
    measurement_loaded = nninfo.analysis.PIDMeasurement.load(experiment=experiment, measurement_id=f'pid_L{layer}')
    pid_summary = get_pid_summary_quantities(measurement_loaded.results)
    nninfo.plot.plot_representational_complexity(pid_summary, ax, label=f'Layer {layer}')

ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))

ax.set_xlabel('Training Epoch')
ax.set_ylabel(r'Repr. Compl. $C$')

ax.legend(bbox_to_anchor=(1, .6), loc='upper left')

ax.set_ylim(1, 4)
ax.set_yticks([1, 2, 3])
ax.set_yticklabels(['1', '2', '3']);

In [ ]:
# Save result
fig.savefig(f"experiments/exp_{experiment_id}/plots/representational_complexity.pdf", bbox_inches='tight');